In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Load Data
train_data = pd.read_csv("train-data.tsv", sep='\t', header=None, names=['label', 'message'])
test_data = pd.read_csv("valid-data.tsv", sep='\t', header=None, names=['label', 'message'])

# Convert labels to binary
train_data['label'] = train_data['label'].map({'ham': 0, 'spam': 1})
test_data['label'] = test_data['label'].map({'ham': 0, 'spam': 1})

In [ ]:
# Preprocess the data
X_train = train_data['message']
y_train = train_data['label']

X_test = test_data['message']
y_test = test_data['label']

# Vectorize text using TF-IDF - Initialize and fit here
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000) # moved out of predict_message to global scope
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Train the model
model = MultinomialNB()
model.fit(X_train_vectorized, y_train)

# Evaluate the model
y_pred = model.predict(X_test_vectorized)
print(f"Model Accuracy: {accuracy_score(y_test, y_pred)}")

# Define the predict_message function
def predict_message(message):
    message_vectorized = vectorizer.transform([message])  # Preprocess input message
    prediction = model.predict(message_vectorized)
    probability = model.predict_proba(message_vectorized)[0][1]
    label = 'spam' if prediction[0] == 1 else 'ham'
    return [probability, label]

In [ ]:
# 1. Import Library
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# 2. Persiapkan Dataset
messages = [
    "how are you doing today",
    "sale today! to stop texts call 98912460324",
    "i dont want to go. can we try it a different day? available sat",
    "our new mobile video service is live. just install on your phone to start watching.",
    "you have won £1000 cash! call to claim your prize.",
    "i'll bring it tomorrow. don't forget the milk.",
    "wow, is your arm alright. that happened to me one time too"
]
labels = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]

# Tambahkan lebih banyak data (opsional untuk memperbaiki performa)
messages += [
    "Earn money fast by completing surveys!",
    "Limited time offer, claim your prize now!",
    "Enjoy our new mobile app with amazing discounts!",
    "Hey, are you free to meet up tomorrow?",
    "Don't forget the meeting at 3 PM today."
]
labels += ["spam", "spam", "spam", "ham", "ham"]

# 3. Preprocessing (TF-IDF Vectorizer)
vectorizer = TfidfVectorizer(stop_words='english', max_features=100, ngram_range=(1, 2))
X = vectorizer.fit_transform(messages)
y = np.array(labels)

# Split dataset untuk training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 4. Pelatihan Model
# a) Naive Bayes
naive_bayes = MultinomialNB(alpha=0.5)
naive_bayes.fit(X_train, y_train)

# b) Logistic Regression
log_reg = LogisticRegression(C=1.0, max_iter=1000)
log_reg.fit(X_train, y_train)

# 5. Evaluasi Model
# Naive Bayes
print("Naive Bayes Results:")
y_pred_nb = naive_bayes.predict(X_test)
print(classification_report(y_test, y_pred_nb))
print(confusion_matrix(y_test, y_pred_nb))

# Logistic Regression
print("\nLogistic Regression Results:")
y_pred_lr = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_lr))
print(confusion_matrix(y_test, y_pred_lr))

# Cross-validation
scores_nb = cross_val_score(naive_bayes, X, y, cv=5, scoring='accuracy')
scores_lr = cross_val_score(log_reg, X, y, cv=5, scoring='accuracy')
print("\nCross-validation Accuracy:")
print(f"Naive Bayes: {np.mean(scores_nb):.2f}")
print(f"Logistic Regression: {np.mean(scores_lr):.2f}")

# 6. Pengujian Model
def predict_message(message):
    vec_message = vectorizer.transform([message])
    nb_pred = naive_bayes.predict(vec_message)[0]
    lr_pred = log_reg.predict(vec_message)[0]
    return {"Naive Bayes": nb_pred, "Logistic Regression": lr_pred}

# Testing dengan beberapa pesan
test_messages = [
    "Congratulations, you've won a free vacation! Call now!",
    "Hey, are we still on for the meeting tomorrow?",
    "Limited time offer! Click here to claim your prize.",
    "Don't forget to pick up groceries on your way home."
]

for msg in test_messages:
    predictions = predict_message(msg)
    print(f"Message: '{msg}' | Naive Bayes: {predictions['Naive Bayes']} | Logistic Regression: {predictions['Logistic Regression']}")


In [ ]:
# Example training data
train_messages = [
    "how are you doing today",
    "sale today! to stop texts call 98912460324",
    "i dont want to go. can we try it a different day? available sat",
    "you have won £1000 cash! call to claim your prize",
    "i'll bring it tomorrow. don't forget the milk."
]
train_labels = ["ham", "spam", "ham", "spam", "ham"]

# Fit the vectorizer and transform training data
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_messages)

# Train the model
model = MultinomialNB()
model.fit(X_train, train_labels)


In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# ... (Your data and model training code from ipython-input-95-2e73ac580f08) ...

# Define the predict_message function
def predict_message(message):
    """
    Predicts the label (spam or ham) of a message.

    Args:
        message: The message string.

    Returns:
        A list containing the predicted label and the probability of being spam.
    """
    # Ensure you're using the SAME vectorizer that was fitted during training
    # This is crucial to maintain feature consistency
    vectorized_message = vectorizer.transform([message])

    # Predict probabilities for spam (class 1) and ham (class 0)
    probability_spam = naive_bayes.predict_proba(vectorized_message)[0][1]  # Using naive_bayes model

    # Adjust the threshold for spam (lowering it to 0.5)
    label = 'spam' if probability_spam > 0.5 else 'ham'

    # Return the prediction and the probability of being spam in a list format
    return [label, probability_spam]

# ... (Your testing code) ...

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
    test_messages = ["how are you doing today",
                     "sale today! to stop texts call 98912460324",
                     "i dont want to go. can we try it a different day? available sat",
                     "our new mobile video service is live. just install on your phone to start watching.",
                     "you have won £1000 cash! call to claim your prize.",
                     "i'll bring it tomorrow. don't forget the milk.",
                     "wow, is your arm alright. that happened to me one time too"
                    ]

    test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
    passed = True

    for msg, ans in zip(test_messages, test_answers):
        prediction = predict_message(msg)
        if prediction[0] != ans:
            print(f"Message: '{msg}' | Predicted: '{prediction[0]}' | Expected: '{ans}'")
            passed = False

    if passed:
        print("You passed the challenge. Great job!")
    else:
        print("You haven't passed yet. Keep trying.")

# Run the test
test_predictions()
